<a href="https://colab.research.google.com/github/Jasper-Hewitt/Wind_project/blob/main/paragraph_similarity_with_sentence_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#input data

In [44]:
!pip install sentence_transformers
#also works with entire alineas 
"""
This is a simple application for sentence embeddings: semantic search

We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.

This script outputs for various queries the top 5 most similar sentences in the corpus.
"""
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('shibing624/text2vec-base-chinese')#('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = ["""2021年末開始大量現身金門、馬祖的澳洲龍蝦，因地緣政治角力、中國抵制澳龍，讓金馬與中國間的走私增加了一個新品項。
早在1990年，中國來的走私船隻就能直接開到金門岸邊上下貨，1992年金馬的戰地政務解除後，走私開始頻繁。2001年小三通開啟，卻存在許多限制，走私仍是日常；甚至，金門貢糖裡的花生不少是來自廈門。但不論是從中國或金馬的角度，長期存在的非法進出口，對各自來說，少了稅收，也不利於食品安全的把關。
是怎麼樣的歷史與地理因素，讓走私日益壯大？金門、馬祖的走私日常會有改變的一天嗎？""", 
"""如今，時代變了。原因當然主要還是美國的經濟停滯、貧富差距、資本主義危機、中產階級衰落⋯⋯，這些社會矛盾甚至威脅了美國的民主制度。
於是自1948年，由美國杜魯門總統主導的GATT，尤其1995年柯林頓政府一手打造人類史上最大的「全球貿易秩序機構」WTO，幾乎已經變成日內瓦的「無用風景」。""",
"""美國是WTO最重要的創始會員國，但現在也是最重要的實施貿易壁壘、最常違反自由貿易的敗訴國之一。
美國目前在WTO的策略是敗訴後再上訴，以曠日費時的訴訟方式，耗費時間，拖垮貿易競爭對手。""",
"""正確地說，人類從歐洲連結至中亞、唐朝、東亞，始於第7世紀，也就是絲路的年代。
那正是中國古典神魔小說《西遊記》幾個主要角色唐僧、孫悟空、豬八戒的時代背景時代。""", 
"""例如11世紀接棒第8世紀的絲路，主導的力量主要來自於歐洲的擴張，尤其義大利諸城邦，威尼斯後來居上。
在二次大戰後，美國主導的全球化之前，人類經濟史已經歷經了至少六大波的全球化，有時候發展神速，大多因為主要佔統治地位的國家出現了快速的擴張，然後因為某個重大危機，全球化的波濤即告一段落。
但嚴格說起來，全球化從來沒有終止，也不會死。它只是暫時中斷了，接著換另一個面貌，另一個產業，重新爬起來。
稍歇一歇，但從未進過墳墓。""",
"""220年前，當大英帝國的君王以向乾隆皇帝祝壽為名義，派出馬戛爾尼勳爵，
分乘五艘船隻經歷十個月的航行，於1793年7月抵達天津大沽口時，他們看到了東方「驕傲而停滯的帝國」，
歷史在那一刻見證了全球化最後的冒險、征服，與從此大英帝國最大的擴張。""",
"""1945年，小羅斯福第四任連任上台沒有多久，突然去世。
美國總統的戰略是延續的，杜魯門除了迫不急待在日本丟了兩顆原子彈，還有一個不可說的任務：終結大英帝國的影響力，取而代之。""",
"""月牙灣是個平坦的沙灘，那些從中國來的走私船隻可以直接開到岸邊上下貨；船隻靠岸前，金門這邊往往會有人先開著怪手前去，
在灘頭挖出一條通道，再用吊車將沙灘上的軌條砦清出一個缺口，方便漁船靠岸。""", 
"""然而「戰地政務」於1992年結束後，軍方開始在金馬地區逐步減少駐軍，連帶導致金馬海防逐漸鬆弛，並在海巡改制進駐之前，為走私船隻提供了走漏洞的機會。
此外，原本由軍方把持調控的物資補給，也逐漸回歸市場機制，而廉價的中國產品，便成了金馬地區日常用品的重要來源，開始以走私方式引入。""",
"""從1986年起，便在金門東門市場經營金紙舖的黃歆凌指出，其實1987年台灣開放兩岸探親之後，
金門的走私貿易就已逐漸興起。她的金紙舖，就面對著金門知名的「大陸街」；全盛時期，整條街上都販賣著來自中國的南北貨、藥材、工藝品。""",
"""這種走私方式，最常見於金門北部礁石密布、海巡巡邏相對困難的海岸線上。
另一個「衝山」走私的重鎮，則是金門東北邊的青嶼和馬山一帶──那裡位在廈門大嶝島正對面，而中國第一個「對台小額貿易市場」，就設在廈門大嶝島上。""",
"""誰也不知道兩岸之間非法進口的行為什麼時候會結束"""




          ]
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['走私', '金門','馬祖']


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(12, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/856 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/319 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]





Query: 走私

Top 5 most similar sentences in corpus:
然而「戰地政務」於1992年結束後，軍方開始在金馬地區逐步減少駐軍，連帶導致金馬海防逐漸鬆弛，並在海巡改制進駐之前，為走私船隻提供了走漏洞的機會。
此外，原本由軍方把持調控的物資補給，也逐漸回歸市場機制，而廉價的中國產品，便成了金馬地區日常用品的重要來源，開始以走私方式引入。 (Score: 0.4229)
2021年末開始大量現身金門、馬祖的澳洲龍蝦，因地緣政治角力、中國抵制澳龍，讓金馬與中國間的走私增加了一個新品項。
早在1990年，中國來的走私船隻就能直接開到金門岸邊上下貨，1992年金馬的戰地政務解除後，走私開始頻繁。2001年小三通開啟，卻存在許多限制，走私仍是日常；甚至，金門貢糖裡的花生不少是來自廈門。但不論是從中國或金馬的角度，長期存在的非法進出口，對各自來說，少了稅收，也不利於食品安全的把關。
是怎麼樣的歷史與地理因素，讓走私日益壯大？金門、馬祖的走私日常會有改變的一天嗎？ (Score: 0.4137)
從1986年起，便在金門東門市場經營金紙舖的黃歆凌指出，其實1987年台灣開放兩岸探親之後，
金門的走私貿易就已逐漸興起。她的金紙舖，就面對著金門知名的「大陸街」；全盛時期，整條街上都販賣著來自中國的南北貨、藥材、工藝品。 (Score: 0.4039)
這種走私方式，最常見於金門北部礁石密布、海巡巡邏相對困難的海岸線上。
另一個「衝山」走私的重鎮，則是金門東北邊的青嶼和馬山一帶──那裡位在廈門大嶝島正對面，而中國第一個「對台小額貿易市場」，就設在廈門大嶝島上。 (Score: 0.3739)
誰也不知道兩岸之間非法進口的行為什麼時候會結束 (Score: 0.3529)
月牙灣是個平坦的沙灘，那些從中國來的走私船隻可以直接開到岸邊上下貨；船隻靠岸前，金門這邊往往會有人先開著怪手前去，
在灘頭挖出一條通道，再用吊車將沙灘上的軌條砦清出一個缺口，方便漁船靠岸。 (Score: 0.3364)
220年前，當大英帝國的君王以向乾隆皇帝祝壽為名義，派出馬戛爾尼勳爵，
分乘五艘船隻經歷十個月的航行，於1793年7月抵達天津大沽口時，他們看到了東方「驕傲而停滯的帝國」，
歷史在那一刻見證了全球化最後的冒險、征服，與從此大英帝國最大的擴張。 (Scor

#calculate accumulated similarity score per keyword

In [45]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [46]:
type(sentence_scores)

dict

In [47]:
#in case we use more or less keywords, we will have to adjust this code slightly
df = pd.DataFrame([[sentence, scores[queries[0]], scores[queries[1]], 
                    scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]] 
                   for sentence, scores in sentence_scores.items()], 
                  columns=['text', 'similarity_score1', 'similarity_score2', 'similarity_score3', 
                           'accumulated_score'])
df = df.sort_values(by='accumulated_score', ascending=False)

In [48]:
#output
df=df.reset_index(drop=True)
df

,text,similarity_score1,similarity_score2,similarity_score3,accumulated_score
0,2021年末開始大量現身金門、馬祖的澳洲龍蝦，因地緣政治角力、中國抵制澳龍，讓金馬與中國間的...,tensor(0.4137),tensor(0.4277),tensor(0.3549),tensor(1.1964)
1,這種走私方式，最常見於金門北部礁石密布、海巡巡邏相對困難的海岸線上。\n另一個「衝山」走私的...,tensor(0.3739),tensor(0.4781),tensor(0.3120),tensor(1.1641)
2,然而「戰地政務」於1992年結束後，軍方開始在金馬地區逐步減少駐軍，連帶導致金馬海防逐漸鬆弛...,tensor(0.4229),tensor(0.3849),tensor(0.3279),tensor(1.1357)
3,從1986年起，便在金門東門市場經營金紙舖的黃歆凌指出，其實1987年台灣開放兩岸探親之後，...,tensor(0.4039),tensor(0.4467),tensor(0.2848),tensor(1.1355)
4,220年前，當大英帝國的君王以向乾隆皇帝祝壽為名義，派出馬戛爾尼勳爵，\n分乘五艘船隻經歷十...,tensor(0.3144),tensor(0.3429),tensor(0.3828),tensor(1.0400)
5,月牙灣是個平坦的沙灘，那些從中國來的走私船隻可以直接開到岸邊上下貨；船隻靠岸前，金門這邊往往...,tensor(0.3364),tensor(0.3750),tensor(0.2378),tensor(0.9492)
6,例如11世紀接棒第8世紀的絲路，主導的力量主要來自於歐洲的擴張，尤其義大利諸城邦，威尼斯後來...,tensor(0.2571),tensor(0.3141),tensor(0.3233),tensor(0.8946)
7,正確地說，人類從歐洲連結至中亞、唐朝、東亞，始於第7世紀，也就是絲路的年代。\n那正是中國古...,tensor(0.2876),tensor(0.2347),tensor(0.3262),tensor(0.8485)
8,誰也不知道兩岸之間非法進口的行為什麼時候會結束,tensor(0.3529),tensor(0.2634),tensor(0.2125),tensor(0.8288)
9,如今，時代變了。原因當然主要還是美國的經濟停滯、貧富差距、資本主義危機、中產階級衰落⋯⋯，這...,tensor(0.2494),tensor(0.2887),tensor(0.2715),tensor(0.8095)


#different method for calculating average (has problems)

In [49]:
#calculate average (This one has some problems)

# #calculate average similarity score 
# # Create a dictionary to store the scores for each sentence
# sentence_scores = {sentence:[] for sentence in corpus}

# # Iterate through the queries and find the top 5 most similar sentences in the corpus
# top_k = min(5, len(corpus))
# for query in queries:
#     query_embedding = embedder.encode(query, convert_to_tensor=True)

#     # We use cosine-similarity and torch.topk to find the highest 5 scores
#     cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
#     top_results = torch.topk(cos_scores, k=top_k)

#     # Iterate through the top results and add the scores to the corresponding sentence in the dictionary
#     for score, idx in zip(top_results[0], top_results[1]):
#         sentence_scores[corpus[idx]].append(score)

# # Create a list of tuples to store the mean scores and sentences
# mean_scores = []

# # Iterate through the dictionary and calculate the mean score for each sentence
# for sentence, scores in sentence_scores.items():
#     if len(scores) == 0:
#         mean_score = -1
#     else:
#         mean_score = sum(scores) / len(scores)
#     mean_scores.append((mean_score, sentence))
# # Sort the list of tuples based on the mean score
# mean_scores.sort(reverse=True)

# # Print the overall similarity overview
# print("Overall similarity overview:")
# for mean_score, sentence in mean_scores:
#     print("Sentence:", sentence)
#     print("Mean score: {:.4f}\n".format(mean_score))

